# Install and setup ollama with models

In [ ]:
!pip install ollama


In [2]:
import os
os.chdir('/kaggle/working/')

#if not os.path.exists('/kaggle/tmp'):
#    os.mkdir('/kaggle/tmp')
#os.chdir('/kaggle/tmp/')

print(os.getcwd())

import subprocess
import os

def run(commands):
    for command in commands:
        with subprocess.Popen(command, shell = True, stdout = subprocess.PIPE, stderr = subprocess.STDOUT, bufsize = 1) as sp:
            for line in sp.stdout:
                line = line.decode("utf-8", errors = "replace")
                if "undefined reference" in line:
                    raise RuntimeError("Failed Processing.")
                print(line, flush = True, end = "")
        pass
    pass
pass

/kaggle/working


In [3]:
import os
os.chdir('/kaggle/working/')

#if not os.path.exists('/kaggle/tmp'):
#    os.mkdir('/kaggle/tmp')
#os.chdir('/kaggle/tmp/')

print(os.getcwd())

import subprocess
import os

def run(commands):
    for command in commands:
        with subprocess.Popen(command, shell = True, stdout = subprocess.PIPE, stderr = subprocess.STDOUT, bufsize = 1) as sp:
            for line in sp.stdout:
                line = line.decode("utf-8", errors = "replace")
                if "undefined reference" in line:
                    raise RuntimeError("Failed Processing.")
                print(line, flush = True, end = "")
        pass
    pass
pass
commands = [
        "curl -fsSL https://ollama.com/install.sh | sh",
]
run(commands)

import os
os.system("/usr/local/bin/ollama serve &")
os.system("echo 'ollama test'")

/kaggle/working


/usr/lib/python3.10/subprocess.py:961: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)


>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle


KeyboardInterrupt: 

## Choosing the model that we will work with

In [ ]:
commands = [
        #"telnet 0.0.0.0 11434"
        # "ollama pull llama3",
        #"ollama pull phi3",
        #"ollama pull mistral",
        "ollama run llama3",
        #"ollama run llama3.1 \"create 10 sentences that ends with apple\""
        # "curl http://127.0.0.1:11434/api/chat -d '{\"model\": \"llama3\", \"stream\": false, \"messages\": [{ \"role\": \"user\", \"content\": \"create 10 sentences that ends with apple\" }]}'"
]
run(commands)

In [ ]:
os.system("curl http://localhost:11434")


# CREW AI TEST

## Edits and something we need to test :

-  Add All domains , try it alone first -> Next step
-  Add RAG System , Try it alone first  -> Next step ( Either rag from crewai or classic rag )
-  Memory setup and its connection to rag ( Context retierver and queue and all that ( it's like another multi agent system )

In [ ]:
!pip install -q crewai langchain_openai langchain 

In [ ]:
pip install 'crewai[tools]'


## SETUP

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool

from crewai.flow import Flow, listen, start
from pydantic import BaseModel, Field, ValidationError
from typing import List, Dict, Optional
import json
import logging
import nest_asyncio
from json import JSONDecodeError
import sys
import json
from typing import List
from pydantic import BaseModel, Field
from crewai import Agent, Task
from crewai.flow.flow import Flow, start, listen

# Initialize LLM 
llm = LLM(model="ollama/llama3", base_url="http://localhost:11434")


domains_above = ["filesystem","users","packages"]


In [ ]:
import json

# Define the RAG knowledge data
rag_knowledge = {
    "filesystem": [
        "Moved ahmed.pdf: mv /home/user/ahmed.pdf /home/user/Documents/",
        "my cat filer is in bla folder"
    ],
    "users": [
        "my password is 213214"
    ],
    "packages": [
        "Installed nginx",
        "installed ollama"
    ]
}

# Define the file path
file_path = 'rag_knowledge.json'

# Write the data to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(rag_knowledge, json_file, indent=4)

## Domain Fetcher Agent

In [ ]:
class DomainAnalysis(BaseModel):
    domains: List[str] = Field(..., description="List of relevant domains")
    confidence: float = Field(..., ge=0, le=1, description="Confidence score between 0 and 1")
    
domain_agent = Agent(
    role="Domain Classifier",
    goal="Categorize Linux prompts into technical domains",
    backstory="Expert in Linux domain classification",
    verbose=True,
    llm=llm
)

domain_task = Task(
    description="Analyze this Linux query: {prompt} and choose domain from the following : {domains}",
    expected_output=DomainAnalysis.schema_json(),
    agent=domain_agent,
    output_json=DomainAnalysis
)


## RAG Agent

In [ ]:
@tool
def rag_retriever(domain: str, query: str) -> str:
    """Retrieves context from domain-specific RAG systems."""
    domain = domain.lower()
    context = rag_knowledge.get(domain)
    return context[0]

class ContextRetrievalResponse(BaseModel):
    context: str = Field(..., description="The retrieved context related to the query")
    
# Context Retrieval Agent
context_agent = Agent(
    role="Technical Context Retriever",
    goal="Fetch relevant technical documentation and examples",
    backstory="Expert in searching and retrieving precise technical information",
    tools=[rag_retriever],
    verbose=True,
    llm=llm
)

context_retrieval_task = Task(
    description="Retrieve context for domain: {domain} related to: {prompt}",
    expected_output=ContextRetrievalResponse.schema_json(),
    agent=context_agent,
    output_json=ContextRetrievalResponse
)

## QA Agent

In [ ]:
class ValidationResult(BaseModel):
    status: str = Field(..., description="Validation status: approved/needs_refinement")
    feedback: str = Field(..., description="Detailed feedback for improvement")


qa_agent = Agent(
    role="Quality Assurance Specialist",
    goal="Ensure context relevance and accuracy",
    backstory="""Meticulous validator with extensive experience in technical 
    quality assurance processes""",
    verbose=True,
    llm=llm
)

context_validation_task = Task(
    description="Validate context quality",
    expected_output=ValidationResult.schema_json(),
    agent=qa_agent,
    output_json=ValidationResult
)

## Code agent

In [ ]:
class CommandOutput(BaseModel):
    main_command: str = Field(..., description="Single Linux command to execute")


command_agent = Agent(
    role="Linux Command Architect",
    goal="Generate safe and effective Linux commands",
    backstory="Experienced system administrator specializing in secure command generation",
    verbose=True,
    llm=llm
)

command_task = Task(
    description="Generate a Linux command for: {prompt} based on these domains: {domains} and the following context : {context}",
    expected_output=CommandOutput.schema_json(),
    agent=command_agent,
    output_json=CommandOutput
)


## Crew 1 -> Test

In [ ]:
# Example crew execution
crew = Crew(
    agents=[domain_agent, command_agent],
    tasks=[domain_task, command_task],
    verbose=True
)

crew_output = crew.kickoff()

# Accessing the crew output
print(f"Raw Output: {crew_output.raw}")
if crew_output.json_dict:
    print(f"JSON Output: {json.dumps(crew_output.json_dict, indent=2)}")
if crew_output.pydantic:
    print(f"Pydantic Output: {crew_output.pydantic}")
print(f"Tasks Output: {crew_output.tasks_output}")
print(f"Token Usage: {crew_output.token_usage}")

## Flow (1) for only Domain and code -> Test

In [ ]:
# Define the flow state
class LinuxFlowState(BaseModel):
    prompt: str = ""
    domains: list = None
    domain_analysis: DomainAnalysis = None
    command_output: CommandOutput = None


class LinuxFlow(Flow[LinuxFlowState]):
    """Flow to execute the agents sequentially"""

    @start()
    def get_user_input(self):
        """Get prompt from the user"""
        self.state.prompt = "list all files in my directory starts with 'a'"
        self.state.domains = domains_above
        return self.state

    @listen(get_user_input)
    def execute_domain_agent(self, state: LinuxFlowState):
        """Execute the domain agent"""
        
        domain_response = domain_agent.execute_task(domain_task,
                context=json.dumps({
                "prompt": state.prompt,
                "domains": ", ".join(state.domains)
                }))
        state.domain_analysis = DomainAnalysis.parse_raw(domain_response)
        
        print(f"\nDomains identified: {state.domain_analysis.domains}")
        print(f"Confidence: {state.domain_analysis.confidence}")
        
        return state

    @listen(execute_domain_agent)
    def execute_command_agent(self, state: LinuxFlowState):
        """Execute the command agent"""
        if not state.domain_analysis:
            print("No domain analysis found. Exiting.")
            return state
        
        domains_str = ", ".join(state.domain_analysis.domains)
        
        command_response = command_agent.execute_task(
            command_task,
            context=json.dumps({
                "prompt": state.prompt,
                "domains": ", ".join(state.domain_analysis.domains)
            }),
            tools=None
        )

        state.command_output = CommandOutput.parse_raw(command_response)
        
        print(f"\nGenerated Command: {state.command_output.main_command}")
        
        return state

## RUN Flow 1

In [ ]:
# async def kickoff():
#     """Run the Linux flow"""
#     await LinuxFlow().kickoff_async()


# # Run the flow asynchronously for compatibility with Jupyter/Colab
# import asyncio
# await kickoff()


## Plot flow 1

In [ ]:
LinuxFlow().plot("guide_creator_flow")


## Flow 2 Code

In [ ]:
# Define the flow state
class LinuxFlowState(BaseModel):
    prompt: str = ""
    domains: list = None
    domain_analysis: DomainAnalysis = None
    command_output: CommandOutput = None
    context_data: dict = None

class LinuxFlow(Flow[LinuxFlowState]):
    """Flow to execute the agents sequentially"""

    @start()
    def get_user_input(self):
        """Get prompt from the user"""
        self.state.prompt = "what is my passward?"
        self.state.domains = domains_above
        return self.state

    @listen(get_user_input)
    def execute_domain_agent(self, state: LinuxFlowState):
        """Execute the domain agent"""
        domain_response = domain_agent.execute_task(domain_task,
                context=json.dumps({
                "prompt": state.prompt,
                "domains": ", ".join(state.domains)
                }))
        state.domain_analysis = DomainAnalysis.parse_raw(domain_response)
        print(f"\nDomains identified: {state.domain_analysis.domains}")
        return state

    @listen(execute_domain_agent)
    def retrieve_context(self, state: LinuxFlowState):
        """Retrieve technical context using RAG"""
        if not state.domain_analysis:
            return state
            
        state.context_data = {}
        for domain in state.domain_analysis.domains:
            context_response = context_agent.execute_task(
                context_retrieval_task,
                context=json.dumps({
                    "domain": domain,
                    "prompt": state.prompt
                })
            )
            state.context_data[domain] = context_response
            
        print("\nRetrieved Context:")
        for domain, context in state.context_data.items():
            print(f"- {domain}: {context[:50]}...")
        return state

    @listen(retrieve_context)
    def execute_command_agent(self, state: LinuxFlowState):
        """Generate command with domain context"""
        if not state.domain_analysis or not state.context_data:
            print("Missing domain analysis or context. Exiting.")
            return state
            
        command_response = command_agent.execute_task(
            command_task,
            context=json.dumps({
                "prompt": state.prompt,
                "domains": ", ".join(state.domain_analysis.domains),
                "context": json.dumps(state.context_data)
            })
        )
        
        state.command_output = CommandOutput.parse_raw(command_response)
        print(f"\nFinal Command: {state.command_output.main_command}")
        return state

In [ ]:
async def kickoff():
    """Run the Linux flow"""
    await LinuxFlow().kickoff_async()


# Run the flow asynchronously for compatibility with Jupyter/Colab
import asyncio
await kickoff()


LinuxFlow().plot("guide_creator_flow")


## Make a fake flow 2 -> what we can do at the end

In [ ]:
from crewai.flow.flow import Flow, listen, router, start, or_
from pydantic import BaseModel


# Define Flow State
class CustomFlowState(BaseModel):
    prompt: str = ""
    domain_output: dict = None
    context_data: dict = None
    qa_data: dict = None
    code_data: dict = None


class flow_fake(Flow[CustomFlowState]):

    @start()
    def user_input(self):
        """User provides a prompt to initiate the flow."""
        self.state.prompt = "User's input prompt here" 
        return "domain_block"

    @listen(user_input)
    def domain_block(self):
        """Processes the user prompt for domain analysis."""
        self.state.domain_output = {"domains": ["example_domain"]}
        return "router"

    @router(or_(domain_block,"context_retriever","qa_block","code_block"))
    def router(self):
        """Routes the flow based on state information."""
        # Example routing logic: Cycle through context, QA, and Code blocks
        if not self.state.context_data:
            return "context_retriever_from_route"
        elif not self.state.qa_data:
            return "qa_block_from_route"
        elif not self.state.code_data:
            return "code_block_from_route"
        else:
            return "finish_from_route"

    # Context Retriever Bidirectional Flow
    @listen(or_("context_retriever_from_route","rag_block"))
    def context_retriever(self):
        """Handles retrieving context and interacting with RAG."""
        self.state.context_data = {"retrieved_info": "sample context data"}
        return "router"  # Return to router

    @listen(context_retriever)
    def rag_block(self):
        """Handles the RAG, Queue, and Stuff."""
        # Simulate RAG processing
        return "context_retriever"

    # QA Bidirectional Flow
    @listen("qa_block_from_route")
    def qa_block(self):
        """Handles question-answering logic."""
        self.state.qa_data = {"qa_response": "sample QA response"}
        return "router"  # Return to router

    # Code Bidirectional Flow
    @listen("code_block_from_route")
    def code_block(self):
        """Handles code generation tasks."""
        self.state.code_data = {"code_output": "sample generated code"}
        return "router"  # Return to router

    # Flow Completion
    @listen("finish_from_route")
    def finish(self):
        """Completes the flow after all tasks are processed."""

    @listen(finish)
    def end(self):
        """Finish the prompt and return what's inside"""
        return self.state


flow = flow_fake()
flow.plot("flow_2")
